In [ ]:
# Import Dependencies

In [ ]:
# Import csv file

In [ ]:
#Import URL and API

In [ ]:
# 1 hour, 6 hour, 24 hour - is there a difference
# Difference between rainy  and snowy day pickups? - Caleb

In [ ]:
# How does the relative temperature and humidity affect pickups? - Caleb

In [ ]:
# Does Wind Speed affect the number of pickups? How does wind with or without precipitation affect it? - Ryan

In [ ]:
#Does different boroughs? Ryan

In [ ]:
# Time of day effect pickups? - Fidel

In [ ]:
#Difference in Weekdays vs Weekends? - James

In [ ]:
#How do Holidays affect pickups? - Fidel Carillo Average holiday pickups vs non holiday with rain and vs non holiday without Difference in Holiday pickups? - Fidel

In [ ]:
# Correlation and Regression

## Analysis